In [1]:

#Load the necessary python libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,roc_curve, auc

In [2]:
'''
P3_X_train = np.loadtxt("P3_Train.csv", delimiter=",")
P3_X_test=np.loadtxt("P3_Test.csv", delimiter=",")
P3_y_train=np.loadtxt("P3_Trainlabel.csv", delimiter=",")
P3_y_test=np.loadtxt("P3_Testlabel.csv", delimiter=",")
'''

P3_X_train = np.loadtxt("Train.csv", delimiter=",")
P3_X_test=np.loadtxt("Test.csv", delimiter=",")
P3_y_train=np.loadtxt("Trainlabel.csv", delimiter=",")
P3_y_test=np.loadtxt("Testlabel.csv", delimiter=",")

In [3]:
X_train=P3_X_train
y_train=P3_y_train

X_test=P3_X_test
y_test=P3_y_test

In [4]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [5]:
import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
params = {}


params['learning_rate'] = .1 # for p1


params['boosting_type'] = 'gbdt'

#params['boosting_type'] = 'goss'

params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
#params['num_leaves'] = 10

params['num_leaves'] = 100  #For P8

params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

/Users/sheuli/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [6]:
y_pred=clf.predict(x_test)
np.savetxt('p3_light_gbm_score_depen.csv',y_pred, delimiter = ',')

In [7]:
y_pred.shape

(2520,)

In [8]:
y_pred=clf.predict(x_test)
#convert into binary values
for i in range(0,2520):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0
    
    
    
np.savetxt('p3_light_gbm_resultlabels.csv',y_pred, delimiter = ',') 

In [9]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

In [10]:
cm

array([[1946,  214],
       [ 116,  244]])

In [11]:
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc

false_positive_rate, recall, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, recall)

In [12]:
roc_auc

0.7893518518518519

In [13]:
TN=cm[0,0]
FP=cm[0,1]
FN=cm[1,0]
TP=cm[1,1]

In [14]:
Sensitivity=(TP/(TP+FN))*100

Specificity=(TN/(TN+FP))*100

recall=(TP/(TP+FN))

Precision = TP/(TP+FP)

Fall_out=(FP/(TN+FP))*100


Fscore=((recall*Precision)/(recall+Precision))*2

PLR=Sensitivity/Fall_out

In [15]:
#Patient 3

Sensitivity,Specificity,Precision, Fall_out,Fscore,PLR

(67.77777777777779,
 90.0925925925926,
 0.5327510917030568,
 9.907407407407408,
 0.5965770171149145,
 6.841121495327103)

In [16]:

from sklearn import metrics
# AUC (Area Under the Curve) を計算する
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
auc = metrics.auc(fpr, tpr)
print(auc)

0.7893518518518519


In [17]:
# ROC曲線をプロット

import matplotlib.pyplot as plt
plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)